# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [3]:
# load data from database
engine = create_engine('sqlite:///ElliottPipeline.db')
conn = engine.connect()
df = pd.read_sql_table('messagesCategorized', conn)
X = df['message']
Y = df.iloc[:,4:] 
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'wordnet'])



#     clean_tokens = []
#     for tok in tokens:
#         clean_tok = lemmatizer.lemmatize(tok).lower().strip()
#         clean_tokens.append(clean_tok)

#     return clean_tokens


def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokenized = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokes = []
    
    for tok in tokenized:
        cleaned = lemmatizer.lemmatize(tok).strip()
        clean_tokes.append(cleaned)
    return clean_tokes

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

In [8]:
pipeline = Pipeline([

        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('rfc', RandomForestClassifier())
])



#      ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ]))

# 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)

In [11]:
pipeline.fit(Xtrain, Ytrain)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

# 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
yhat = pipeline.predict(Xtest)

In [12]:
for i in range(len(Ytest.columns)):
    print("Performance for {} category".format(Ytest.columns[i]))
    print(classification_report(Ytest.iloc[:,i], yhat[:,i]))

Performance for related category
             precision    recall  f1-score   support

          0       0.57      0.45      0.50      1484
          1       0.85      0.90      0.87      5061

avg / total       0.78      0.80      0.79      6545

Performance for request category
             precision    recall  f1-score   support

          0       0.89      0.98      0.94      5430
          1       0.84      0.42      0.56      1115

avg / total       0.88      0.89      0.87      6545

Performance for offer category
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        20

avg / total       0.99      1.00      1.00      6545

Performance for aid_related category
             precision    recall  f1-score   support

          0       0.70      0.91      0.79      3862
          1       0.77      0.44      0.56      2683

avg / total       0.73      0.72      0.70      6545

Perfor

In [13]:
preclist = []
reclist = []
f1list = []
for i in range(len(Ytest.columns)):
    preclist.append(precision_score(Ytest.iloc[:,i], yhat[:,i]))
    reclist.append(recall_score(Ytest.iloc[:,i], yhat[:,i]))
    f1list.append(f1_score(Ytest.iloc[:,i], yhat[:,i]))

In [14]:
print("Average Precision: ", np.around(np.mean(preclist),3))
print("Average Recall: ", np.around(np.mean(reclist),3))
print("Average F1: ", np.around(np.mean(f1list),3))

Average Precision:  0.523
Average Recall:  0.113
Average F1:  0.154


# 6. Improve your model
Use grid search to find better parameters. 

In [44]:
pipeline.get_params()['vect']

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7fc23bea47b8>, vocabulary=None)

In [45]:
pipeline.get_params()['tfidf']

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [46]:
pipeline.get_params()['rfc']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [47]:
from sklearn.model_selection import GridSearchCV
parameters = [{
#     'vect__binary': [True, False],
#     'vect__max_features' : [None, 100, 1000],
    'tfidf__norm' : ['l1', 'l2'],
#     'tfidf__use_idf' : [True, False],
#     'moc__n_jobs' : [1, 5, 10],
#     'moc__estimator__p' : [1, 2]
}]

cv = GridSearchCV(pipeline, param_grid=parameters)

In [48]:
cv.fit(Xtrain, Ytrain)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'tfidf__norm': ['l1', 'l2']}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [49]:
yhat_cv = cv.predict(Xtest)

# 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [51]:
for i in range(len(Ytest.columns)):
    print("Performance for {} category".format(Ytest.columns[i]))
    print(classification_report(Ytest.iloc[:,i], yhat_cv[:,i]))

Performance for related category
             precision    recall  f1-score   support

          0       0.57      0.46      0.51      1501
          1       0.85      0.90      0.87      5044

avg / total       0.78      0.80      0.79      6545

Performance for request category
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5458
          1       0.88      0.36      0.51      1087

avg / total       0.89      0.89      0.87      6545

Performance for offer category
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6503
          1       0.00      0.00      0.00        42

avg / total       0.99      0.99      0.99      6545

Performance for aid_related category
             precision    recall  f1-score   support

          0       0.69      0.92      0.79      3842
          1       0.79      0.43      0.55      2703

avg / total       0.73      0.72      0.69      6545

Perfor

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [98]:
cvPreclist = []
cvReclist = []
cvF1list = []
for i in range(len(Ytest.columns)):
    cvPreclist.append(precision_score(Ytest.iloc[:,i], yhat_cv[:,i]))
    cvReclist.append(recall_score(Ytest.iloc[:,i], yhat_cv[:,i]))
    cvF1list.append(f1_score(Ytest.iloc[:,i], yhat_cv[:,i]))

In [94]:
print("Average Precision: ", np.around(np.mean(cvPreclist),3))
print("Average Recall: ", np.around(np.mean(cvReclist),3))
print("Average F1: ", np.around(np.mean(cvF1list),3))

Average Precision:  0.604
Average Recall:  0.108
Average F1:  0.15


# 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [75]:
pipeline_imp = Pipeline([

        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('moc', MultiOutputClassifier(RandomForestClassifier()))
])


In [76]:
pipeline_imp.fit(Xtrain, Ytrain)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [77]:
yhat_imp = pipeline_imp.predict(Xtest)

In [78]:
for i in range(len(Ytest.columns)):
    print("Performance for {} category".format(Ytest.columns[i]))
    print(classification_report(Ytest.iloc[:,i], yhat_imp[:,i]))

Performance for related category
             precision    recall  f1-score   support

          0       0.63      0.37      0.47      1501
          1       0.83      0.94      0.88      5044

avg / total       0.79      0.81      0.79      6545

Performance for request category
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5458
          1       0.85      0.40      0.54      1087

avg / total       0.88      0.89      0.87      6545

Performance for offer category
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6503
          1       0.00      0.00      0.00        42

avg / total       0.99      0.99      0.99      6545

Performance for aid_related category
             precision    recall  f1-score   support

          0       0.73      0.88      0.79      3842
          1       0.75      0.53      0.62      2703

avg / total       0.74      0.73      0.72      6545

Perfor

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [85]:
from sklearn.metrics import precision_score, recall_score

In [95]:
impPreclist = []
impReclist = []
impF1list = []
for i in range(len(Ytest.columns)):
    impPreclist.append(precision_score(Ytest.iloc[:,i], yhat_imp[:,i]))
    impReclist.append(recall_score(Ytest.iloc[:,i], yhat_imp[:,i]))
    impF1list.append(f1_score(Ytest.iloc[:,i], yhat_imp[:,i]))
    
# np.mean(impf1list)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [96]:
print("Average Precision: ", np.around(np.mean(impPreclist),3))
print("Average Recall: ", np.around(np.mean(impReclist),3))
print("Average F1: ", np.around(np.mean(impF1list),3))

Average Precision:  0.581
Average Recall:  0.164
Average F1:  0.22


# 9. Export your model as a pickle file

In [105]:
import pickle
pickle.dump(pipeline_imp, open('picklefile_finalModel.sav', 'wb'))

In [106]:
# pickle.dump(pipeline, open('picklefile_noCV.sav', 'wb'))
# pickleParty = pickle.load(open('/home/workspace/picklefile_noCV.sav', 'rb'))
# pickleParty.predict(['help I need drinking water down by the church', 'my dog has gone missing I need to find him'])

array([[ 1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [50]:
# pickle.dump(cv, open('picklefile_CV.sav', 'wb'))

In [11]:
# pickle.load(open('picklefile_finalModel.sav', 'rb'))

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.